## 2.4.1 日本語テキストの分類

chapter1, chapter2で学んできたことを用いて 日本語テキストのカテゴリをランダムフォレスト で分類します。
ここでもlivedoor newsを用います。ランダムフォレストに与えるデータはベクトル表現化したニュース記事で、9種類のカテゴリーに分類します。
記事をベクトルで表すことにより、教師あり学習で学んだ方法をそのまま適用して記事の分類が行えます。

このチャプターの学習フローは以下のようになります。

・livedoor newsの読み込みと分類：「コーパスの取り出し」

・データをトレイニングデータとテストデータに分割する：機械学習概論の「ホールドアウト法の理論と実践」

・tf-idfでトレイニングデータとテストデータをベクトル化する：「BOW tf-idfによる重み付け（実装）」、「fit関数」

・ランダムフォレストで学習：教師あり分類の「ランダムフォレスト」

・実装：「コーパスのカテゴリをランダムフォレストで実装」

・精度を上げる：「精度をあげる」

## 2.4.2 fit関数

scikit-learn の変換系クラス(StandardScaler、Normalizer、TfidfVectorizer など)には、 fit(), fit_transform(), transform() などの関数があります。

・fit()関数 ：渡されたデータの統計（最大値、最小値、平均、など）を取得して、メモリに保存。

・transform()関数 ：fit()で取得した情報を用いてデータを書き換える。

・fit_transform()関数 ：fit()の後にtransform()を実施する。

fit()関数 はトレーニングデータセットからパラメーターを学習するために使用され、 tranform()関数 は　学習したパラメーターに基づいてデータが再形成されます。

つまり、トレーニングデータの場合は fit_transform関数 を用い、
テストデータの場合は,トレーニングデータの fit() の結果に基づくので、 transform()関数 を行う必要があります。
fit()させるのはトレーニングデータです。テストデータにはfitさせてはいけません。

## 2.4.3 コーパスのカテゴリをランダムフォレストで実装

それではこれまで学んできたことを用いて、 livedoornewsコーパスのカテゴリをランダムフォレストで分類しましょう。
「日本語テキストの分類」でも書いたようにフローは以下のようになります。

・livedoor newsの読み込みと分類：「コーパスの取り出し」

・データをトレイニングデータとテストデータに分割する：機械学習概論の「ホールドアウト法の理論と実践」

・tf-idfでトレイニングデータとテストデータをベクトル化する：「BOW tf-idfによる重み付け（実装）」、「fit関数」

・ランダムフォレストで学習：教師あり分類の「ランダムフォレスト」

・実装：「コーパスのカテゴリをランダムフォレストで実装」

・精度を上げる：「精度をあげる」

トレーニングデータにはfit_transformを、テストデータにはtransformを使用しましょう。

In [10]:
import glob
import random
from sklearn.model_selection import train_test_split
from janome.tokenizer import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier


def load_livedoor_news_corpus():
 
    category = {
        "dokujo-tsushin": 1,
        "it-life-hack":2,
        "kaden-channel": 3,
        "livedoor-homme": 4,
        "movie-enter": 5,
        "peachy": 6,
        "smax": 7,
        "sports-watch": 8,
        "topic-news":9
    }
    docs  = []
    labels = []

    for c_name, c_id in category.items():
        files = glob.glob("./text/{c_name}/{c_name}*.txt".format(c_name=c_name))

#         text = ""
        for file in files:
            with open(file, "r", encoding="utf-8") as f:
                lines = f.read().splitlines() 
#                 url = lines[0]  
#                 datetime = lines[1]  
                subject = lines[2]
                body = "".join(lines[3:])
                text = subject + body

            docs.append(text)
            labels.append(c_id)

    return docs, labels

docs, labels = load_livedoor_news_corpus()

# データをトレイニングデータとテストデータに分割する（機械学習概論 「ホールドアウト法の理論と実践」)
train_data, test_data, train_labels, test_labels = train_test_split(docs, labels, test_size=0.2, random_state=0)

# tf-idfでトレイニングデータとテストデータをベクトル化する。(「fit関数」)
#-------------------------------------------------------

vectorizer = TfidfVectorizer()
train_matrix = vectorizer.fit_transform(train_data) # train_dataをベクトル化
test_matrix = vectorizer.transform(test_data) # test_dataをベクトル化

#-------------------------------------------------------


# ランダムフォレストで学習（教師あり分類　「ランダムフォレスト」)
clf = RandomForestClassifier(n_estimators=2)
clf.fit(train_matrix, train_labels)



# 精度の出力
print(clf.score(train_matrix, train_labels))
print(clf.score(test_matrix, test_labels))


0.908535550653
0.71302578019


## 2.4.4 精度を上げる

「コーパスのカテゴリをランダムフォレストで実装」で実装したプログラムの精度を上げる作業をしたいと思います。
TfidfVectorizer() のパラメーターに tokenizer=関数 を設定すると、指定した関数でテキストを分割することができます。
例えば、以下の関数を tokenizer= の引数とすると、「名詞、動詞、形容詞、形容動詞」のみの分割されたテキストを用います。
そのため、分析に必要ない助詞・助動詞等がないので精度が上がる事になります。

```python

def tokenize(text):
    tokens = t.tokenize(",".join(text))
    noun = []
    for token in tokens:
        partOfSpeech = token.part_of_speech.split(",")[0]
 
        if partOfSpeech == "名詞":
            noun.append(token.surface)        
        if partOfSpeech == "動詞":        
            noun.append(token.surface)
        if partOfSpeech == "形容詞":
            noun.append(token.surface)        
        if partOfSpeech == "形容動詞":        
            noun.append(token.surface)            
    return noun
```


In [9]:
import glob
import random
from sklearn.model_selection import train_test_split
from janome.tokenizer import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier


def load_livedoor_news_corpus():
 
    category = {
        "dokujo-tsushin": 1,
        "it-life-hack":2,
        "kaden-channel": 3,
        "livedoor-homme": 4,
        "movie-enter": 5,
        "peachy": 6,
        "smax": 7,
        "sports-watch": 8,
        "topic-news":9
    }
    docs  = []
    labels = []

    for c_name, c_id in category.items():
        files = glob.glob("./text/{c_name}/{c_name}*.txt".format(c_name=c_name))

#         text = ""
        for file in files:
            with open(file, "r", encoding="utf-8") as f:
                lines = f.read().splitlines() 
#                 url = lines[0]  
#                 datetime = lines[1]  
                subject = lines[2]
                body = "".join(lines[3:])
                text = subject + body

            docs.append(text)
            labels.append(c_id)

    return docs, labels

docs, labels = load_livedoor_news_corpus()

# データをトレイニングデータとテストデータに分割する（機械学習概論 「ホールドアウト法の理論と実践」)
train_data, test_data, train_labels, test_labels = train_test_split(docs, labels, test_size=0.2, random_state=0)

# tf-idfでトレイニングデータとテストデータをベクトル化する。(「fit関数」)
#-------------------------------------------------------

# 品詞分類用の関数
def tokenize(text):
    tokens = t.tokenize(",".join(text))
    noun = []
    for token in tokens:
    # 品詞を取り出し
        partOfSpeech = token.part_of_speech.split(",")[0]
 
        if partOfSpeech == "名詞":
            noun.append(token.surface)        
        if partOfSpeech == "動詞":        
            noun.append(token.surface)
        if partOfSpeech == "形容詞":
            noun.append(token.surface)        
        if partOfSpeech == "形容動詞":        
            noun.append(token.surface)            
    return noun

vectorizer = TfidfVectorizer(tokenize)
train_matrix = vectorizer.fit_transform(train_data) # train_dataをベクトル化
test_matrix = vectorizer.transform(test_data) # test_dataをベクトル化

#-------------------------------------------------------


# ランダムフォレストで学習（教師あり分類　「ランダムフォレスト」)
clf = RandomForestClassifier(n_estimators=2)
clf.fit(train_matrix, train_labels)



# 精度の出力
print(clf.score(train_matrix, train_labels))
print(clf.score(test_matrix, test_labels))

0.912947564908
0.728629579376


In [8]:
import glob
import random
from sklearn.model_selection import train_test_split
from janome.tokenizer import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier


def load_livedoor_news_corpus():
 
    category = {
        "dokujo-tsushin": 1,
        "it-life-hack":2,
        "kaden-channel": 3,
        "livedoor-homme": 4,
        "movie-enter": 5,
        "peachy": 6,
        "smax": 7,
        "sports-watch": 8,
        "topic-news":9
    }
    docs  = []
    labels = []

    for c_name, c_id in category.items():
        files = glob.glob("./text/{c_name}/{c_name}*.txt".format(c_name=c_name))

#         text = ""
        for file in files:
            with open(file, "r", encoding="utf-8") as f:
                lines = f.read().splitlines() 
#                 url = lines[0]  
#                 datetime = lines[1]  
                subject = lines[2]
                body = "".join(lines[3:])
                text = subject + body

            docs.append(text)
            labels.append(c_id)

    return docs, labels

docs, labels = load_livedoor_news_corpus()

# データをトレイニングデータとテストデータに分割する（機械学習概論 「ホールドアウト法の理論と実践」)
train_data, test_data, train_labels, test_labels = train_test_split(docs, labels, test_size=0.2, random_state=0)

# tf-idfでトレイニングデータとテストデータをベクトル化する。(「fit関数」)
#-------------------------------------------------------

# 品詞分類用の関数
def tokenize(text):
    tokens = t.tokenize(",".join(text))
    noun = []
    for token in tokens:
    # 品詞を取り出し
        partOfSpeech = token.part_of_speech.split(",")[0]
 
        if partOfSpeech == "名詞":
            noun.append(token.surface)        
        if partOfSpeech == "動詞":        
            noun.append(token.surface)
#         if partOfSpeech == "形容詞":
#             noun.append(token.surface)        
#         if partOfSpeech == "形容動詞":        
#             noun.append(token.surface)            
    return noun

vectorizer = TfidfVectorizer(tokenize)
train_matrix = vectorizer.fit_transform(train_data) # train_dataをベクトル化
test_matrix = vectorizer.transform(test_data) # test_dataをベクトル化

#-------------------------------------------------------


# ランダムフォレストで学習（教師あり分類　「ランダムフォレスト」)
clf = RandomForestClassifier(n_estimators=2)
clf.fit(train_matrix, train_labels)



# 精度の出力
print(clf.score(train_matrix, train_labels))
print(clf.score(test_matrix, test_labels))

0.897166129306
0.730664857531
